In [1]:
import pandas as pd
import numpy as np


In [ ]:
											
data = pd.read_csv(
    "./weekly_sales/transactional data/sales_11_5_to_1-06.csv")
data
data['BusinessDate'] = pd.to_datetime(data['BusinessDate'])
data[['NetAmount'	,'Quantity']]=data[['NetAmount'	,'Quantity']].astype(float)
print(data.BusinessDate.min())
print(data.BusinessDate.max() )


In [29]:
daily_data = {
    'OrderDate': pd.to_datetime([
        
        '2024-06-03', # Monday
        '2024-06-04', # Tuesday
        '2024-06-05', # Wednesday
        '2024-06-06', # Thursday
        '2024-06-07', # Friday
        '2024-06-08', # Saturday
        '2024-06-09', # Sunday
        '2024-06-10', # Monday (start of next week)
        '2024-06-11'  # Tuesday
    ]),
    'Revenue': [100, 120, 90, 150, 200, 80, 70, 130, 110]
}
daily_sales_df = pd.DataFrame(daily_data)
daily_sales_df


,OrderDate,Revenue
0,2024-06-03,100
1,2024-06-04,120
2,2024-06-05,90
3,2024-06-06,150
4,2024-06-07,200
5,2024-06-08,80
6,2024-06-09,70
7,2024-06-10,130
8,2024-06-11,110


In [30]:
daily_sales_df['OrderDate'] = pd.to_datetime(daily_sales_df['OrderDate'])
daily_sales_df = daily_sales_df.set_index('OrderDate')
daily_sales_df


,Revenue
OrderDate,
2024-06-03,100
2024-06-04,120
2024-06-05,90
2024-06-06,150
2024-06-07,200
2024-06-08,80
2024-06-09,70
2024-06-10,130
2024-06-11,110


In [32]:

weekly_revenue = daily_sales_df['Revenue'].resample('W').sum()
weekly_revenue_df = weekly_revenue.reset_index()
weekly_revenue_df.columns = ['WeekStartDate', 'WeeklyRevenue']
weekly_revenue_df


,WeekStartDate,WeeklyRevenue
0,2024-06-09,810
1,2024-06-16,240


In [33]:
weekly_revenue_df["Week"] = weekly_revenue_df['WeekStartDate'].dt.weekday
weekly_revenue_df


AttributeError: 'RangeIndex' object has no attribute 'to_period'

In [36]:


data[['BusinessDate', 'NetAmount', 'OrderId']].set_index('BusinessDate').resample('W').agg({'NetAmount':'sum','OrderId':'nunique'})



,NetAmount,OrderId
BusinessDate,,
2025-05-18,790036.73,29308
2025-05-25,823678.68,30131
2025-06-01,824431.93,30051


In [ ]:
data["Week"] = data.index.to_period("W-MON").start_time
    # Group by Week and OrderPlacedMethod
weekly_summary = (
        data.groupby(["Week"])
        .agg({"Quantity": "sum", "OrderId": pd.Series.nunique, "NetAmount": "sum"})
        .rename(columns={"OrderId": "UniqueOrders"})
    )
weekly_summary


In [7]:
# daily_summary=pd.read_csv('./weekly_sales/daily_summary.csv',parse_dates=['Date'])
# daily_summary.head()
# daily_summary.Date.max()
# daily_summary


In [11]:
daily_sale=pd.read_csv('./weekly_sales/daily_sales.csv',parse_dates=['Date'])
daily_sale.Date.max()
daily_sale['brand']='LC'
daily_sale.head(2)


,FranchiseStore,OrderPlacedMethod,MenuItemName,Date,UniqueOrderCount,Revenue,AvgOrderValue,brand
0,01389-00001,DoorDash,NaN,2025-05-19,1,0.00,0.00,LC
1,01389-00001,DoorDash,2 L Beverage,2025-05-19,3,13.71,4.57,LC


In [15]:
daily_sale['week']=np.where(daily_sale['Date']<='2025-05-27','prev','recent')


In [17]:
agg = daily_sale.groupby(['brand', 'week']).agg({
    'UniqueOrderCount': 'sum',
    'Revenue': 'sum'
}).reset_index()

agg['AvgOrderValue'] = agg['Revenue'] / agg['UniqueOrderCount']
agg


,brand,week,UniqueOrderCount,Revenue,AvgOrderValue
0,LC,prev,161036,1006668.63,6.251202
1,LC,recent,99073,641441.98,6.474438


In [148]:
bundle_analysis=pd.read_csv('./weekly_sales/bundle.csv',parse_dates=['Date'])
bundle_analysis.head()


,FranchiseStore,OrderPlacedMethod,Date,ItemType,UniqueOrderCount,Revenue,AvgOrderValue
0,01389-00001,DoorDash,2025-05-19,Crazy Combo,1,4.81,4.81000
1,01389-00001,DoorDash,2025-05-19,Regular,20,404.35,20.21750
2,01389-00001,DoorDash,2025-05-20,Crazy Combo,2,9.62,4.81000
3,01389-00001,DoorDash,2025-05-20,Regular,16,320.02,20.00125
4,01389-00001,DoorDash,2025-05-21,Crazy Combo,4,19.24,4.81000


In [149]:
# we had 1 order for crazy combo at frechise x mode y date z, it gave us revenue 4.81,
# does a single order contaons combo and regular ones?


np.int64(75961)

In [ ]:
daily_sale


In [ ]:
daily_sale


In [143]:
a=daily_sale[(daily_sale['FranchiseStore']=='01389-00001')&(daily_sale['OrderPlacedMethod']=='DoorDash')&
           ((daily_sale['Date']>='2025-05-19' ) & (daily_sale['Date']<='2025-05-25' )) ]


In [146]:
a['UniqueOrderCount'].sum()
# item is ordered in unique id orders


np.int64(382)

In [145]:
a['UniqueOrderCount'].sum()


np.int64(382)

In [139]:


weekly_lc = (
    daily_sale
    .groupby(['FranchiseStore', 'OrderPlacedMethod'])
    .resample('W-MON', on='Date')[['UniqueOrderCount', 'Revenue', 'AvgOrderValue']]
    .sum()
    .reset_index()
)
weekly_lc


,FranchiseStore,OrderPlacedMethod,Date,UniqueOrderCount,Revenue,AvgOrderValue
0,01389-00001,DoorDash,2025-05-19,45,409.16,253.600000
1,01389-00001,DoorDash,2025-05-26,384,3224.23,1340.423864
2,01389-00001,DoorDash,2025-06-02,347,2960.72,1296.353722
3,01389-00001,Grubhub,2025-05-19,2,12.64,12.640000
4,01389-00001,Grubhub,2025-05-26,5,32.15,32.150000
...,...,...,...,...,...,...
918,03646-00004,UberEats,2025-05-26,1,15.85,15.850000
919,03646-00004,UberEats,2025-06-02,4,30.20,30.200000
920,03646-00004,Website,2025-05-19,43,214.60,151.335000
921,03646-00004,Website,2025-05-26,201,1122.73,827.333333


In [ ]:
01389-00001
DoorDash
2025-05-19
157




In [127]:
daily_sale['Date']


FranchiseStore        object
OrderPlacedMethod     object
MenuItemName          object
Date                  object
UniqueOrderCount       int64
Revenue              float64
AvgOrderValue        float64
dtype: object

In [9]:
vibe=pd.read_csv('vibe.csv')
vibe.head()


,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue
0,01389-00001,Grubhub,2024-02-26,16.12,3PO,LC,16.12
1,01389-00001,Mobile,2024-02-26,6109.86,1PO,LC,6109.86
2,01389-00001,UberEats,2024-02-26,359.55,3PO,LC,359.55
3,01389-00001,DoorDash,2024-02-26,2017.23,3PO,LC,2017.23
4,01389-00001,Website,2024-02-26,2767.06,1PO,LC,2767.06


In [12]:
vibe[vibe['Brand']=='LC']['WeekStartDate'].max()
vibe[vibe['Brand']=='WS']['WeekStartDate'].max()


'2025-03-10'

In [30]:
vibe['OrderMode'].replace({"Online":'Website'},inplace=True)


In [111]:
previous_week='2025-03-03'
week='2025-03-10'
weeks=[previous_week,week]


In [113]:
last_two_weeks=vibe[vibe['WeekStartDate'].isin(weeks)]	
last_two_weeks.WeekStartDate.unique()


array(['2025-03-03', '2025-03-10'], dtype=object)

In [32]:
# Channel	Total_Orders	Total_Revenue	Avg_Order_Value	Total_Refunds	Total_Discounts		
last_week=vibe[vibe['WeekStartDate']==week]																		
last_week.groupby(['Brand','OrderMode'])['WeeklyNetAmount'].sum().reset_index()


,Brand,OrderMode,WeeklyNetAmount
0,LC,DoorDash,157200.50
1,LC,Grubhub,5084.56
2,LC,Mobile,363610.83
3,LC,UberEats,30292.64
4,LC,Website,192351.98
5,WS,DoorDash,684729.93
6,WS,UberEats,209982.45
7,WS,Website,988874.66


In [33]:

# Group to get revenue by Week, Brand, and OrderMode
grouped = last_week.groupby(["WeekStartDate", "Brand", "OrderMode"]).agg(
    WeeklyNetAmount=("WeeklyNetAmount", "sum")
).reset_index()

# Add total weekly revenue per brand (for revenue share calculation)
grouped["TotalByBrandWeek"] = grouped.groupby(["WeekStartDate", "Brand"])["WeeklyNetAmount"].transform("sum")

# Add Weekly revenue for the digital channel (it's same as WeeklyNetAmount at this granularity, but for clarity)
grouped["WeeklyChannelRevenue"] = grouped["WeeklyNetAmount"]

# Calculate revenue share %
grouped["RevenueSharePct"] = round(100 * grouped["WeeklyNetAmount"] / grouped["TotalByBrandWeek"], 2)

# Sort for presentation
grouped = grouped.sort_values(by=["WeekStartDate", "Brand", "RevenueSharePct"], ascending=[True, True, False])

# Optional: sort for easier reading
grouped = grouped.sort_values(by=["WeekStartDate", "Brand", "RevenueSharePct"], ascending=[True, True, False])
grouped. sort_values(by=['Brand','RevenueSharePct'],ascending=False)


,WeekStartDate,Brand,OrderMode,WeeklyNetAmount,TotalByBrandWeek,WeeklyChannelRevenue,RevenueSharePct
7,2025-03-10,WS,Website,988874.66,1883587.04,988874.66,52.50
5,2025-03-10,WS,DoorDash,684729.93,1883587.04,684729.93,36.35
6,2025-03-10,WS,UberEats,209982.45,1883587.04,209982.45,11.15
2,2025-03-10,LC,Mobile,363610.83,748540.51,363610.83,48.58
4,2025-03-10,LC,Website,192351.98,748540.51,192351.98,25.70
0,2025-03-10,LC,DoorDash,157200.50,748540.51,157200.50,21.00
3,2025-03-10,LC,UberEats,30292.64,748540.51,30292.64,4.05
1,2025-03-10,LC,Grubhub,5084.56,748540.51,5084.56,0.68


In [35]:
import plotly.express as px
# Format percentages to 2 decimals for text labels
grouped['revenue_text'] = grouped['RevenueSharePct'].apply(lambda x: f'{x:.2f}%')
brand_colors = {
    'LC': 'rgb(246, 35, 151)',  
    'WS': 'rgb(22, 198, 206)'  
}
brand_order_modes = grouped.groupby("Brand")["OrderMode"].unique().to_dict()

fig = px.bar(
    grouped,
    x="OrderMode",
    y="RevenueSharePct",
    color="Brand",
    facet_col="Brand",
    barmode="group",
    text="revenue_text",
    color_discrete_map=brand_colors,
    title="Revenue Percentage by Channel and Brand",
    category_orders={
        "OrderMode": sorted(grouped["OrderMode"].unique()),  # this is global
        "Brand": sorted(grouped["Brand"].unique())
    }
)


fig.update_traces(textposition='outside')

fig.update_layout(
    xaxis_title='Channel',
    yaxis_title='Revenue (%)',
    yaxis=dict(range=[1, 100])
)
fig.update_layout(
    width=800,   # Set the desired width (in pixels)
    height=500,  # Set the desired height (in pixels)
)

fig.show()


In [39]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get unique brands
brands = grouped["Brand"].unique()

# Initialize subplot
fig = make_subplots(
    rows=1, cols=len(brands),
    subplot_titles=brands,
    shared_yaxes=True
)

# Add traces for each brand
for i, brand in enumerate(brands):
    df_brand = grouped[grouped["Brand"] == brand]
    fig.add_trace(
        go.Bar(
            x=df_brand["OrderMode"],
            y=df_brand["RevenueSharePct"],
            text=df_brand["revenue_text"],
            textposition="outside",
            marker_color=brand_colors.get(brand, "gray"),
            name=brand,
            showlegend=False
        ),
        row=1, col=i+1
    )

# Layout settings
fig.update_layout(
    title="Revenue Share by Channel)",
    yaxis_title="Revenue (%)",
    height=500,
    width=800,
    uniformtext_minsize=7,
    uniformtext_mode='show'
)
fig.write_image('Revenue Share.png')
fig.show()


In [41]:
last_week.groupby(['Brand'])['StoreID'].nunique()


Brand
LC    72
WS    74
Name: StoreID, dtype: int64

In [16]:
last_week.to_csv('week_2025-03-10.csv')


In [43]:
last_week.head()


,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue
251,01389-00001,Mobile,2025-03-10,6753.66,1PO,LC,6753.66
252,01389-00001,Grubhub,2025-03-10,57.99,3PO,LC,57.99
253,01389-00001,DoorDash,2025-03-10,3007.84,3PO,LC,3007.84
254,01389-00001,UberEats,2025-03-10,228.32,3PO,LC,228.32
255,01389-00001,Website,2025-03-10,3113.38,1PO,LC,3113.38


In [50]:
# Step 1: Group by Brand and Store to get total revenue per store
store_rev = last_week.groupby(['Brand', 'StoreID'])['WeeklyNetAmount'].sum().reset_index()

# Step 2: Compute total revenue per Brand
brand_total = store_rev.groupby('Brand')['WeeklyNetAmount'].transform('sum')

# Step 3: Calculate revenue share per store within brand
store_rev['RevenueSharePct'] = (store_rev['WeeklyNetAmount'] / brand_total) * 100

store_rev.sort_values(by='RevenueSharePct',ascending=False)


,Brand,StoreID,WeeklyNetAmount,RevenueSharePct
143,WS,84,46901.82,2.490027
135,WS,448,45726.36,2.427621
43,LC,03222-00015,17965.57,2.400080
53,LC,03222-00026,17934.57,2.395938
23,LC,01940-00010,17852.60,2.384988
...,...,...,...,...
130,WS,2884,11713.46,0.621870
41,LC,03222-00009,4579.15,0.611744
42,LC,03222-00010,4456.45,0.595352
126,WS,2851,10437.94,0.554152


In [51]:
fig = px.violin(
    store_rev,
    x='Brand',
    y='RevenueSharePct',
    box=True,  # Add embedded box plot
    points='all',  # Show all data points
    title='Revenue Share % Distribution by Brand',
    color='Brand',
    color_discrete_map=brand_colors
)
fig.update_layout(
    yaxis_title='Revenue Share (%)',
    xaxis_title='Brand',
    height=500,
    width=600
)
fig.show()



In [57]:

# Step 1: Calculate 75th percentile for each brand
q75 = store_rev.groupby('Brand')['RevenueSharePct'].transform(lambda x: x.quantile(0.85))

# Step 2: Filter stores above 75th percentile within their brand
top_stores = store_rev[store_rev['RevenueSharePct'] > q75]


In [58]:
fig = px.violin(
    top_stores,
    x='Brand',
    y='RevenueSharePct',
    box=True,  # Add embedded box plot
    points='all',  # Show all data points
    title='Revenue Share % Distribution by Brand',
    color='Brand',
    color_discrete_map=brand_colors
)
fig.update_layout(
    yaxis_title='Revenue Share (%)',
    xaxis_title='Brand',
    height=500,
    width=600
)
fig.show()


In [87]:
top_stores.sort_values(by=['Brand','StoreID','RevenueSharePct'],ascending=False,inplace=True)


/var/folders/lg/b8pw7x296y397cn0g302yt680000gp/T/ipykernel_72299/309156422.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [88]:
import plotly.express as px

fig = px.box(
    store_rev,  # use full dataset
    x="Brand",
    y="RevenueSharePct",
    points="all",
    color="Brand",
    title="Revenue Share Distribution per Brand with All Stores"
)

# Highlight top stores in a second trace
fig.add_trace(
    px.scatter(
        top_stores,
        x="Brand",
        y="RevenueSharePct",
        color_discrete_sequence=["black"],
        hover_data=["StoreID", "WeeklyNetAmount"]
    ).data[0]
)

fig.update_layout(yaxis_title="Revenue Share (%)")
fig.show()


In [109]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Unique brands
brands = top_stores['Brand'].unique()

# Create subplot layout
fig = make_subplots(
    rows=1, cols=len(brands),
    subplot_titles=brands,
    shared_yaxes=False
)

# Add bar plots for each brand
for i, brand in enumerate(brands):
    brand_df = top_stores[top_stores['Brand'] == brand].sort_values(by='RevenueSharePct', ascending=True)

    fig.add_trace(
        go.Bar(
            x=brand_df['RevenueSharePct'],
            y=brand_df['StoreID'].astype(str),
            orientation='h',
            text=brand_df['RevenueSharePct'].round(1).astype(str) + '%',
            textfont=dict(size=10),
            textposition='outside',
            marker=dict(
                color=brand_colors.get(brand, 'gray'),
                line=dict(width=0.5, color='black')  # Slight border for thinner bars
            ),
            width=0.5,  # Reduce bar thickness
            name=brand,
            showlegend=False
        ),
        row=1, col=i + 1
    )

# Layout tweaks
fig.update_layout(
    height=400,  # Increased from 500
    width=1200,
    title_text="Top 25% Stores by Revenue Share per Brand.",
    margin=dict(t=100, l=10, r=10, b=50),  # More top margin
)

# Label each y-axis
for i in range(len(brands)):
    fig['layout'][f'yaxis{i+1}']['title'] = 'Store ID'
    break
# Label each y-axis
for i in range(len(brands)):
    fig['layout'][f'xaxis{i+1}']['title'] = 'Store ID'
    break

fig.write_image('store_performace.png')
fig.show()


In [ ]:
# item performance
# Current week	Previous Week	Digital Channel	Volume Trend WoW	Item Volume (Last Week) 	Item Volume (Current Week)	WoW Volume % Change 																					


In [114]:
last_two_weeks


,StoreID,OrderMode,WeekStartDate,WeeklyNetAmount,channel,Brand,revenue
246,01389-00001,Grubhub,2025-03-03,27.15,3PO,LC,27.15
247,01389-00001,DoorDash,2025-03-03,3119.65,3PO,LC,3119.65
248,01389-00001,Mobile,2025-03-03,8400.13,1PO,LC,8400.13
249,01389-00001,Website,2025-03-03,4260.65,1PO,LC,4260.65
250,01389-00001,UberEats,2025-03-03,232.66,3PO,LC,232.66
...,...,...,...,...,...,...,...
29312,2961,DoorDash,2025-03-10,10131.70,3PO,WS,10131.70
29316,2961,Website,2025-03-03,17276.03,1PO,WS,17276.03
29317,2961,Website,2025-03-10,16966.91,1PO,WS,16966.91
29323,2961,UberEats,2025-03-03,1956.58,3PO,WS,1956.58


In [115]:
last_two_weeks.columns


Index(['StoreID', 'OrderMode', 'WeekStartDate', 'WeeklyNetAmount', 'channel',
       'Brand', 'revenue'],
      dtype='object')